In [1]:
!pip install keras-tuner

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):
    model=keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('fil_1',min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('fil_2',min_value=32,max_value=128,step=16),
            kernel_size=hp.Choice('kern_2',values=[3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_unit_1',min_value=32,max_value=128,step=16),
            activation='relu'
        ),
        keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [9]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output4',project_name="MnistFashion")

INFO:tensorflow:Reloading Tuner from output4\MnistFashion\tuner0.json


In [10]:
tuner_search.search(train_images,train_labels,epochs=1,validation_split=0.1)

Trial 5 Complete [00h 03m 44s]
val_accuracy: 0.0925000011920929

Best val_accuracy So Far: 0.8931666612625122
Total elapsed time: 00h 14m 49s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 80)        23120     
                                                                 
 flatten (Flatten)           (None, 38720)             0         
                                                                 
 dense (Dense)               (None, 64)                2478144   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,502,746
Trainable params: 2,502,746
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_images,train_labels,epochs=3,validation_split=0.1,initial_epoch=1)

Epoch 2/3
1688/1688 [==============================] - 163s 96ms/step - loss: 0.2510 - accuracy: 0.9077 - val_loss: 0.2502 - val_accuracy: 0.9040
Epoch 3/3
1688/1688 [==============================] - 166s 98ms/step - loss: 0.1937 - accuracy: 0.9272 - val_loss: 0.2462 - val_accuracy: 0.9137


acc=0.9137